In [1]:
# Code adapted from "PHAS0007 Script for session 5: Notebook 1 (of 2)",(Dash and Lemos, 2019) {3}
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
Hides code for easier viewing, Toggle code boxes <a href="javascript:code_toggle()">here</a>.''')

# Neutrino event classification

This mini-project's dataset is comprised of a number of small files containing images of simulated neutrino interactions in a hypothetical detector that looks an awful lot like the detectors of the NOvA experiment. For each neutrino interaction the images consist of two $100 \times 80$ pixel images that represent the $x \times z$ and $y \times z$ projections of the tracks of particles in the detector.

The data for this mini-project comes in the form of the following files:

| File | Description |
| ----------- | ----------- |
| neutrino1.h5 | The 1st HDF5 file containing event images and meta deta |
| $\vdots$ | The middle ones |
| neutrino200.h5| The 200th HDF5 file|


The images show the energy deposited by simulated neutrinos in a NOvA like detector. Some of the meta information in the hdf5 file is described below

| Label | Description |
| ----------- | ----------- |
| neutrino/nuenergy | Neutrino Energy (GeV) |
| neutrino/lepenergy | Lepton Energy (GeV) |
| neutrino/finalstate | Interaction |
| neutrino/finalstate | Final State |
 

The [PDG code](https://pdg.lbl.gov/2019/reviews/rpp2019-rev-monte-carlo-numbering.pdf) is a number which identifies the particle type (e.g electron=11, electron-neutrino=12, etc.)

The $interaction$ says what kind of interaction occured and is defined in the enumeration below.

## Machine learning tasks
1. Develop a machine learning classifier that can successfully identify $\nu_\mu$ charged-current events
2. Test your machine learning classifier and investigate how the efficiency of the classifier depends on the meta data variables shown above

### Potential extensions
1. Write a machine learning algorithm to determine the energy of the neutrino
2. Write a machine learning algorithm to determine the flavour of the neutrino
3. Write a machine learning algorithm to determine $y=$ lepton energy over neutrino energy
4. Write a machine learning algorithm to determine the number of protons or pions
5. Write a machine learning algorithm to determine the interaction mode.




In [ ]:
# Import for processing data files
import h5py

# Standard imports
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd

# Added for a progress bar
from tqdm import tqdm

# TensorFlow and tf.keras imports
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers.merge import concatenate

# Import for analysising model predicted results
from sklearn import metrics

In [ ]:
# Classes encoding the interactions and states

import enum 
class Interaction(enum.Enum):
    kNumuQE =0           # Numu CC QE interaction
    kNumuRes =1           # Numu CC Resonant interaction
    kNumuDIS = 2          # Numu CC DIS interaction
    kNumuOther = 3        # Numu CC, other than above
    kNueQE = 4            # Nue CC QE interaction
    kNueRes = 5           # Nue CC Resonant interaction
    kNueDIS = 6           # Nue CC DIS interaction
    kNueOther = 7         # Nue CC, other than above
    kNutauQE = 8          # Nutau CC QE interaction
    kNutauRes = 9         # Nutau CC Resonant interaction
    kNutauDIS =10         # Nutau CC DIS interaction
    kNutauOther =11       # Nutau CC, other than above
    kNuElectronElastic = 12# NC Nu On E Scattering
    kNC =13                # NC interaction
    kCosmic =14           # Cosmic ray background
    kOther =15            # Something else.  Tau?  Hopefully we don't use this
    kNIntType=16          # Number of interaction types, used like a vector size

    
class FinalState(enum.Enum):
    kNumu0tr0sh=0          # Numu CC - no track no shower
    kNumu0tr1sh=1          # Numu CC - no track  1 shower
    kNumu0tr2sh=enum.auto()          # Numu CC - no track  2 shower
    kNumu0trMsh=enum.auto()          # Numu CC - no track 3+ shower
    kNumu1tr0sh=enum.auto()          # Numu CC -  1 track no shower
    kNumu1tr1sh=enum.auto()          # Numu CC -  1 track  1 shower
    kNumu1tr2sh=enum.auto()          # Numu CC -  1 track  2 shower
    kNumu1trMsh=enum.auto()          # Numu CC -  1 track 3+ shower
    kNumu2tr0sh=enum.auto()          # Numu CC -  2 track no shower
    kNumu2tr1sh=enum.auto()          # Numu CC -  2 track  1 shower
    kNumu2tr2sh=enum.auto()          # Numu CC -  2 track  2 shower
    kNumu2trMsh=enum.auto()          # Numu CC -  2 track 3+ shower
    kNumuMtr0sh=enum.auto()          # Numu CC - 3+ track no showe
    kNumuMtr1sh=enum.auto()          # Numu CC - 3+ track  1 shower
    kNumuMtr2sh=enum.auto()          # Numu CC - 3+ track  2 showe
    kNumuMtrMsh=enum.auto()          # Numu CC - 3+ track 3+ shower
    kNue0tr0sh=enum.auto()           # Nue CC - no track no shower
    kNue0tr1sh=enum.auto()           # Nue CC - no track  1 shower
    kNue0tr2sh=enum.auto()           # Nue CC - no track  2 showe
    kNue0trMsh=enum.auto()           # Nue CC - no track 3+ shower
    kNue1tr0sh=enum.auto()           # Nue CC -  1 track no shower
    kNue1tr1sh=enum.auto()           # Nue CC -  1 track  1 shower
    kNue1tr2sh=enum.auto()           # Nue CC -  1 track  2 shower
    kNue1trMsh=enum.auto()           # Nue CC -  1 track 3+ shower
    kNue2tr0sh=enum.auto()           # Nue CC -  2 track no shower
    kNue2tr1sh=enum.auto()           # Nue CC -  2 track  1 shower
    kNue2tr2sh=enum.auto()           # Nue CC -  2 track  2 shower
    kNue2trMsh=enum.auto()           # Nue CC -  2 track 3+ shower
    kNueMtr0sh=enum.auto()           # Nue CC - 3+ track no shower
    kNueMtr1sh=enum.auto()           # Nue CC - 3+ track  1 shower
    kNueMtr2sh=enum.auto()           # Nue CC - 3+ track  2 shower
    kNueMtrMsh=enum.auto()           # Nue CC - 3+ track 3+ shower
    kNC0tr0sh=enum.auto()           # NC CC - no track no shower
    kNC0tr1sh=enum.auto()           # NC CC - no track  1 shower
    kNC0tr2sh=enum.auto()           # NC CC - no track  2 shower
    kNC0trMsh=enum.auto()           # NC CC - no track 3+ shower
    kNC1tr0sh=enum.auto()           # NC CC -  1 track no shower
    kNC1tr1sh=enum.auto()           # NC CC -  1 track  1 shower
    kNC1tr2sh=enum.auto()           # NC CC -  1 track  2 shower
    kNC1trMsh=enum.auto()           # NC CC -  1 track 3+ shower
    kNC2tr0sh=enum.auto()           # NC CC -  2 track no shower
    kNC2tr1sh=enum.auto()           # NC CC -  2 track  1 shower
    kNC2tr2sh=enum.auto()           # NC CC -  2 track  2 shower
    kNC2trMsh=enum.auto()           # NC CC -  2 track 3+ shower
    kNCMtr0sh=enum.auto()           # NC CC - 3+ track no shower
    kNCMtr1sh=enum.auto()           # NC CC - 3+ track  1 shower
    kNCMtr2sh=enum.auto()           # NC CC - 3+ track  2 shower
    kNCMtrMsh=enum.auto()           # NC CC - 3+ track 3+ shower
    kCosmicFS=enum.auto()           # Cosmic ray background
    kOtherFS=enum.auto()            # Something else.  Tau?  Hopefully we don't use this
    kNFStType=enum.auto()            # Number of interaction types, used like a vector size



In [ ]:
# import the urllib library
import urllib.request

# Create empty dictionary to hold data files
data = {}

# Loops to store data in dictionaries
for i in range(1,30):
    
    # Copy a network object to a local file
    urllib.request.urlretrieve('http://www.hep.ucl.ac.uk/undergrad/0056/other/projects/nova/neutrino'+str(i)+'.h5', 
                               'neutrino'+str(i)+'.h5')
    
    # Open the local h5 file with h5py, stores in dictionary
    data[i] = h5py.File('neutrino'+str(i)+'.h5','r')
    
    # Can use exec to store as variables 
    #exec(f"df{i} = h5py.File('neutrino{i}.h5','r')")

In [ ]:
df = data[1]
i  = 10

# Print neutrino data meta data values
print(Interaction(df['neutrino']['interaction'][i]))
print(FinalState(df['neutrino']['finalstate'][i]))
print(df['neutrino']['evt'][i])

# Plot meta data as a table
pd.DataFrame(df['neutrino'])

In [ ]:
#Print the keys in the neutrino meta data
print(df.keys())
print(df['neutrino'].keys())

#Get an numpy array containing the event image, and reshape it from flat to 2x100x80
print(np.shape(df['cvnmap']))
print(df['neutrino']['evt'])

In [ ]:
# Extract event images to plot
event0 = np.array(df['cvnmap'][0]).reshape((2,100,80))

# Plot the first event, look it is a nice long muon track
fig, ax = plt.subplots(1,2, figsize = (15,5))

im1 = ax[0].imshow(event0[0].T, cmap = 'Reds')
ax[0].set_title('X-Z track image')
ax[0].set_xlabel('x, arb')
ax[0].set_ylabel('z, arb')

im2 = ax[1].imshow(event0[1].T, cmap = 'Reds')
ax[1].set_title('Y-Z track image')
ax[1].set_xlabel('y, arb')
ax[1].set_ylabel('z, arb')

# fig.colorbar(im1, ax= ax[0])
# fig.colorbar(im2, ax= ax[1])
plt.show()

In [ ]:
print("Neutrino Final State code",df['neutrino']['finalstate'][3])
print("Interaction was ",Interaction(df['neutrino']['cycle'][7]))
print("Neutrino energy",df['neutrino']['finalstateprong'][5][0],"GeV")

## Preliminary Code:

### Functions:
- Creating functions to help process and hold data

In [ ]:
def Classes(data, labels):
    ''' Divides data into classes in a dictionary.
    
    Inputs:
        data   - Data array containg images 
        labels - Corresponding class labels for images

    Return:
        classes - Dictionary of class stratafied data, in which keys are the classes 
        and values are corresponding images
    '''
    
    # Initialise dictionary
    classes = {}
    
    # for the index and values in the array
    for i, v in enumerate(labels):
        
        # Ensures values are integers not arrays
        if type(v) == np.ndarray:
            value = v[0]
        else:
            value = v
            
        # Adds value as key of dictionary and attach corresponding data
        if value in list(classes.keys()):
            classes[value].append(data[i])

        else:
            classes[value] = [data[i]] 
   
    return classes

In [ ]:
def ClassEqualised(data, labels):
    ''' Ensures the arrays have equal class distributions.
    
    Input:
        data   - Data array containg images 
        labels - Corresponding class labels for images

    Return:
        CE_data   - Data array containg images of equal class distribution  
        CE_labels - Corresponding class labels for images of equal class distribution
    
    '''
    # Extract Classes from data
    classes = Classes(data, labels)
    
    # Random shuffle of data
    for keys, values in list(classes.items()):
        p = np.random.permutation(len(values))
        classes[keys] = np.array(values)[p] 
    
    # Find size of smallest class
    N = np.min([len(classes[i]) for i in list(classes.keys())])
    
    # List comprehension to equally join each classes back to one array
    CE_data   = np.array([ i for group in list(classes.values()) for i in group[:N] ])
    CE_labels = np.array([ [k] for i in list(classes.keys()) for k in [i]*N ])
    
    # Randomly shuffle arrays, preserving labeling
    p = np.random.permutation(len(CE_labels))
    CE_data   = CE_data[p]
    CE_labels = CE_labels[p]
    
    return CE_data, CE_labels

In [ ]:
def Dictionary(Input_dictionary, Input_array, data_no, **bin_kwargs):
    ''' Function to sort data in array into dictionary holding according images for each class 
    and subclass of meta data.
    
    Inputs:
    Input_dictionary - Dictionary to hold  meta data strata
    Input_array      - Array of data (images, labels, or predictions), ordered in corresponding order of data[data_no]
    data_no          - Number of the data file corresponding to input array order 
    **bin_kwargs     - Keyword arguements used sort energy in binned strata
    
    Return:
    Input_dictionary - Updated dictionary holding sorted data

    '''
    
    # For each meta data key in file
    for i in data[data_no]['neutrino'].keys():
        
        # Ensures meta data key is in dictionary
        if i in Input_dictionary:
            None
        else:
            Input_dictionary[i] = {}
        
        # For the data in the array adds to correct 'sub-class' in meta data
        for j in range(len(Input_array)):
            
            # Extract data 'sub-class' 
            k = data[data_no]['neutrino'][i][j][0]
            
            # Adds data to corresponding dictionary location
            if k in Input_dictionary[i]:
                Input_dictionary[i][k].append(Input_array[j]) 
            else: 
                Input_dictionary[i][k] = [Input_array[j]]
           
                
    # Adds key for energy into dictionary
    if 'energy' in Input_dictionary:
        None
    else:
        Input_dictionary['energy'] = {} 

    # For the data in the array adds to corresponding energy key
    for i in range(len(Input_array)):
        
        # Calculates corresponding image energy
        k = data[data_no]['neutrino']['nuenergy'][i][0] + data[data_no]['neutrino']['lepenergy'][i][0]

        # Adds data to corresponding dictionary location
        if k in Input_dictionary['energy']:
            Input_dictionary['energy'][k].append(Input_array[i])

        else: 
            Input_dictionary['energy'][k] = [Input_array[i]]
     
    
    # If we have bin_kwargs, adds to dictionary
    for keys, values in bin_kwargs.items():
        
        # Extract data label
        bin_data_label = str(keys)+str(values)
        
        # Adds label to dictionary
        if bin_data_label in Input_dictionary:
            None
        else:
            Input_dictionary[bin_data_label] = {} 
            
        # For the data in the array adds to corresponding key
        for i in range(len(Input_array)):
             # Uses kwarg value to round data into stara bin, with condition for if it is energy
            if keys == 'energy':
                ke= data[data_no]['neutrino']['nuenergy'][i][0] + data[data_no]['neutrino']['lepenergy'][i][0]
                k = values*round(ke/values)
            else:
                k = values*round(data[data_no]['neutrino'][keys][i][0]/values)
            
            # Adds data to corresponding dictionary location
            if k in Input_dictionary[bin_data_label]:
                Input_dictionary[bin_data_label][k].append(Input_array[i])

            else: 
                Input_dictionary[bin_data_label][k] = [Input_array[i]]

    return Input_dictionary

### Normalisation Tests: 
(Note: Kernal runs out of memory later if you run this section, you have to restart kernal and run code without this section to run testing later)
- Exploring that image values vs corresponding energies and effect of normalsation; intend to check if there is underlying correlation in pixel values and if any information is lost from normalisation.

In [ ]:
# Extract event images
event  = np.array(data[1]['cvnmap'], dtype = 'float')
event_inputs = event.reshape((event.shape[0],2,100,80)).transpose(0,2,3,1)

# Extract event images and normalise
event_norm  = np.array(data[1]['cvnmap'], dtype = 'float')
event_norm /= event_norm.max()
event_norm /= event_norm.max()
event_inputs_norm = event_norm.reshape((event_norm.shape[0],2,100,80)).transpose(0,2,3,1)

# Extract event images and alternativenormalise
event_norm2  = np.array(data[1]['cvnmap'], dtype = 'float')
event_norm2 /= (event_norm2.max(axis=1)[:,None])
event_norm2 /= (event_norm2.max(axis=1)[:,None])
event_inputs_norm2 = event_norm2.reshape((event_norm2.shape[0],2,100,80)).transpose(0,2,3,1)

# Create meta data dictionary
Inputs_dic       =  Dictionary({}, event_inputs,  1)
Inputs_dic_norm  =  Dictionary({}, event_inputs_norm, 1)
Inputs_dic_norm2 =  Dictionary({}, event_inputs_norm2, 1)

In [ ]:
# Extract Energy data keys
key, key_norm, key_norm2, = (np.array(list(Inputs_dic['nuenergy'].keys())), 
                             np.array(list(Inputs_dic_norm['nuenergy'].keys())),
                             np.array(list(Inputs_dic_norm2['nuenergy'].keys()))  )

# Initialise arrays to hold pixel values
ave, ave_norm, ave_norm2 = np.zeros([len(key),2]), np.zeros([len(key),2]), np.zeros([len(key),2])
mxm, mxm_norm, mxm_norm2 = np.zeros([len(key),2]), np.zeros([len(key),2]), np.zeros([len(key),2])

# Take max and mean value for each corresponding energy
for i in range(len(Inputs_dic['nuenergy'])):   
    ave[i][1], ave[i][0] = np.mean(Inputs_dic['nuenergy'][key[i]]), key[i]
    mxm[i][1], mxm[i][0] = np.max(Inputs_dic['nuenergy'][key[i]]),  key[i]
      
# Take max and mean value for each corresponding energy, for normalised data
for i in range(len(Inputs_dic_norm['nuenergy'])): 
    ave_norm[i][1], ave_norm[i][0] = np.mean(Inputs_dic_norm['nuenergy'][key_norm[i]]), key_norm[i]    
    mxm_norm[i][1], mxm_norm[i][0] = np.max(Inputs_dic_norm['nuenergy'][key_norm[i]]),  key_norm[i]

# Take max and mean value for each corresponding energy, for normalised 2 data
for i in range(len(Inputs_dic_norm2['nuenergy'])): 
    ave_norm2[i][1], ave_norm2[i][0] = np.mean(Inputs_dic_norm2['nuenergy'][key_norm2[i]]), key_norm2[i]    
    mxm_norm2[i][1], mxm_norm2[i][0] = np.max(Inputs_dic_norm2['nuenergy'][key_norm2[i]]),  key_norm2[i]
    

In [ ]:
# Plot corresponding plots of max/mean pixel value against energy
fig, ax = plt.subplots(3,2, figsize=(20,10))

ax[0,0].scatter(ave[:,0],ave[:,1], color= 'k')
ax[0,0].set_title('Average pixel value against energy for un-normalised data')
ax[0,0].set_xlabel('Energy, eV')
ax[0,0].set_ylabel('Average pixel value, arb')

ax[0,1].scatter(mxm[:,0],mxm[:,1], color= 'k')
ax[0,1].set_title('Maximum pixel value against energy for un-normalised data')
ax[0,1].set_xlabel('Energy, eV')
ax[0,1].set_ylabel('Maximum pixel value, arb')

ax[1,0].scatter(ave_norm[:,0],ave_norm[:,1], color= 'k')
ax[1,0].set_ylim(-0.001, 0.008)
ax[1,0].set_title('Average pixel value against energy for batch normalised data')
ax[1,0].set_xlabel('Energy, eV')
ax[1,0].set_ylabel('Average pixel value, arb')

ax[1,1].scatter(mxm_norm[:,0],mxm_norm[:,1], color= 'k')
ax[1,1].set_title('Maximum pixel value against energy for batch normalised data')
ax[1,1].set_xlabel('Energy, eV')
ax[1,1].set_ylabel('Maximum pixel value, arb')

ax[2,0].scatter(ave_norm2[:,0],ave_norm2[:,1], color= 'k')
ax[2,0].set_ylim(-0.001, 0.015)
ax[2,0].set_title('Average pixel value against energy for individually normalised data')
ax[2,0].set_xlabel('Energy, eV')
ax[2,0].set_ylabel('Average pixel value, arb')

ax[2,1].scatter(mxm_norm2[:,0],mxm_norm2[:,1], color= 'k')
ax[2,1].set_title('Maximum pixel value against energy for individually normalised data')
ax[2,1].set_xlabel('Energy, eV')
ax[2,1].set_ylabel('Maximum pixel value, arb')


plt.subplots_adjust(hspace = 0.7)
plt.show()

In [ ]:
# Initialising arrays
maxpix = np.zeros(len(event))
avepix = np.zeros(len(event))

maxpix_norm = np.zeros(len(event_norm))
avepix_norm = np.zeros(len(event_norm))

# Storing average and maximum over
for i in range(len(event_norm)):
    maxpix[i]= np.max(event_inputs[i])
    avepix[i]= np.mean(event_inputs[i])
    
    maxpix_norm[i]= np.max( event_inputs_norm[i])
    avepix_norm[i]= np.mean(event_inputs_norm[i])

# Extract energy arrays from data
energy   = np.array(data[1]['neutrino']['nuenergy']) + np.array(data[1]['neutrino']['lepenergy'])
nuenergy = np.array(data[1]['neutrino']['nuenergy'])
lepenergy= np.array(data[1]['neutrino']['lepenergy'])

In [ ]:
# Plotting Histograms of data
fig, ax = plt.subplots(3,2, figsize=(15,15))

ax[0,0].hist(nuenergy, bins=30, color= 'k')
ax[0,0].set_title('Histogram of image Nu-Energies')
ax[0,0].set_xlabel('Energy, eV')
ax[0,0].set_ylabel('No. of Images')

ax[0,1].hist(lepenergy, bins=30, color= 'k')
ax[0,1].set_title('Histogram of image Lep-Energies')
ax[0,1].set_xlabel('Energy, eV')
ax[0,1].set_ylabel('No. of Images')

ax[1,0].hist(maxpix, bins=30, color= 'k')
ax[1,0].set_title('Histogram of Maximum Pixel Value')
ax[1,0].set_xlabel('Pixel value, arb')
ax[1,0].set_ylabel('No. of Images')

ax[1,1].hist(avepix, bins=30, color= 'k')
ax[1,1].set_title('Histogram of Average Pixel Value')
ax[1,1].set_xlabel('Pixel value, arb')
ax[1,1].set_ylabel('No. of Images')

ax[2,0].hist(maxpix_norm, bins=30, color= 'k')
ax[2,0].set_title('Histogram of Maximum Normalised Pixel Value')
ax[2,0].set_xlabel('Pixel value, arb')
ax[2,0].set_ylabel('No. of Images')

ax[2,1].hist(avepix_norm, bins=30, color= 'k')
ax[2,1].set_title('Histogram of Average Normalised Pixel Value')
ax[2,1].set_xlabel('Pixel value, arb')
ax[2,1].set_ylabel('No. of Images')

plt.show()

#### Comment:
- We can see that normalising images removes shape to the the distibutions with respect to pixel value, so underlying information over the group of images as a whole could be lost; any gains in computational speed would be negated, as the relative pixel brightness/ variation may hold information on energy of interaction; to ensure underlying correlation in results is preserved we will normalise by maximum value over whole batch, not individual images.

##### Images of Interaction types:

In [ ]:
index = 34

#Plot the event from training data
fig, ax = plt.subplots(1,3, figsize=(20,5))
im1 = ax[0].imshow(Inputs_dic['interaction'][0][index][:,:,0].T, cmap = 'Reds')
ax[0].set_title('QE interaction track image')
ax[0].set_xlabel('z, arb')
ax[0].set_ylabel('x, arb')

im2 = ax[1].imshow(Inputs_dic['interaction'][1][index][:,:,0].T, cmap = 'Reds')
ax[1].set_title('Res interaction track image')
ax[1].set_xlabel('z, arb')
ax[1].set_ylabel('x, arb')

im3 = ax[2].imshow(Inputs_dic['interaction'][2][index][:,:,0].T, cmap = 'Reds')
ax[2].set_title('DIS interaction track image')
ax[2].set_xlabel('z, arb')
ax[2].set_ylabel('x, arb')

# fig.colorbar(im1, ax= ax[0])
# fig.colorbar(im2, ax= ax[1])
# fig.colorbar(im3, ax= ax[2])
plt.show()

In [ ]:
index = 3
print(Inputs_dic['interaction'].keys())
#Plot the event from training data
fig, ax = plt.subplots(1,3, figsize=(20,5))
im1 = ax[0].imshow(Inputs_dic['interaction'][0][index][:,:,0].T, cmap = 'Reds')
ax[0].set_title('CC NuMu interaction track image')
ax[0].set_xlabel('z, arb')
ax[0].set_ylabel('x, arb')

im2 = ax[1].imshow(Inputs_dic['interaction'][4][index][:,:,0].T, cmap = 'Reds')
ax[1].set_title('CC NuE interaction track image')
ax[1].set_xlabel('z, arb')
ax[1].set_ylabel('x, arb')

im3 = ax[2].imshow(Inputs_dic['interaction'][13][index][:,:,0].T, cmap = 'Reds')
ax[2].set_title('CC NuTau interaction track image')
ax[2].set_xlabel('z, arb')
ax[2].set_ylabel('x, arb')

# fig.colorbar(im1, ax= ax[0])
# fig.colorbar(im2, ax= ax[1])
# fig.colorbar(im3, ax= ax[2])
plt.show()

### Model Architecture Exploration:

#### Create Training Data:
Concatenate data to create larger batch. Here we have to use Class Equalisation function as imported data containes unbalanced classes, as shown in cell before balancing there are 88% ones and only 12% zeros; this causes miss fitting and stagnation of models as the network can reduce lost by predicting all ones and fall into false minima.

In [ ]:
# Extract initial data and label information (use uint8 to useless memory)
event  = np.array(data[1]['cvnmap'], dtype='uint8')
event_l= np.array(data[1]['neutrino']['interaction'],dtype='uint8')

# Loop to join data into arrays to get larger batch
for i in range(2,10):
    event  = np.concatenate(( event   , np.array(data[i]['cvnmap'], dtype = 'uint8') ))  
    event_l= np.concatenate(( event_l , np.array(data[i]['neutrino']['interaction'], dtype='uint8') )) 

# Normalise images
# event /= event.max()

# Reshape inputs into correct shape for networks
event_inputs = event.reshape((event.shape[0],2,100,80)).transpose(0,2,3,1)

# Format labels such that Numu are 1, and rest are 0
event_labels= np.where( event_l > 3, 0, 1).astype(bool)

In [ ]:
# Create class dictionary
classes = Classes(event_inputs, event_labels)

# Extract number of each class
a = len(list(classes.values())[0])
b = len(list(classes.values())[1])

# Print percentage of each class
print('Shape of data', event_inputs.shape)
print('Percentage of', list(classes.keys())[0], 'is', a/(a+b))
print('Percentage of', list(classes.keys())[1], 'is', b/(a+b))

In [ ]:
# Equalise class data
train_event_inputs, train_event_labels = ClassEqualised(event_inputs, event_labels)
print('Shape of training data', train_event_inputs.shape)

# Create class dictionary
classes = Classes(train_event_inputs, train_event_labels)

# Extract number of each class
a = len(list(classes.values())[0])
b = len(list(classes.values())[1])

# Print percentage of each class
print('Percentage of after class equalising', list(classes.keys())[0], 'is', a/(a+b))
print('Percentage of after class equalising', list(classes.keys())[1], 'is', b/(a+b))

In [ ]:
# Checking images are preserved
index = 14

#Plot the event from training data
fig, ax = plt.subplots(1,2, figsize=(15,5))
im1 = ax[0].imshow(train_event_inputs[index][:,:,0].T, cmap = 'Reds')
ax[0].set_title('X-Z track image')
ax[0].set_xlabel('z, arb')
ax[0].set_ylabel('x, arb')

im2 = ax[1].imshow(train_event_inputs[index][:,:,1].T, cmap = 'Reds')
ax[1].set_title('Y-Z track image')
ax[1].set_xlabel('z, arb')
ax[1].set_ylabel('y, arb')

# fig.colorbar(im1, ax= ax[0])
# fig.colorbar(im2, ax= ax[1])
plt.show()

#### Model 1:
Seperable conv


In [ ]:
# Setting up Convolutional neural network
test_model1 = keras.models.Sequential()

# First, Seperable convolutional layer of 32, 7x7 kernals; Pooling 2x2 and Spatial dropout for overfitting
test_model1.add(keras.layers.SeparableConv2D(32, 7, activation='relu', input_shape=(100,80,2)))
test_model1.add(keras.layers.MaxPooling2D((2, 2), strides = 2))
test_model1.add(keras.layers.SpatialDropout2D(0.3))

# Second convolutional layer of 32, 3x3 kernals; Pooling 2x2  and dropout for overfitting
test_model1.add(keras.layers.Conv2D(64, 3, activation='relu', input_shape=(100,80,2)))
test_model1.add(keras.layers.MaxPooling2D((2, 2), strides = 2))
test_model1.add(keras.layers.Dropout(0.3))

# model.add(keras.layers.SeparableConv2D(128, 3, activation='relu', input_shape=(100,80,2)))
# model.add(keras.layers.MaxPooling2D((2, 2), strides = 1))

# Third convolutional layer of 64, 3x3 kernals 
test_model1.add(keras.layers.Conv2D(64, 3, activation='relu'))
test_model1.add(keras.layers.Dropout(0.7))

# Fully connected Neural Network
test_model1.add(keras.layers.Flatten())
test_model1.add(keras.layers.Dense(128, activation='relu'))
test_model1.add(keras.layers.Dropout(0.7))
test_model1.add(keras.layers.Dense(64, activation='relu'))

# We have a dropout layer to prevent overfitting
test_model1.add(keras.layers.Dropout(0.7))
test_model1.add(keras.layers.Dense(1, activation = 'sigmoid'))

# Compile the network with binary crossentropy loss and adam optimiser with learning rate of 1.0
test_model1.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.001), 
              loss      = tf.keras.losses.BinaryCrossentropy(from_logits=False), 
              metrics   = [keras.metrics.BinaryAccuracy()] )

test_model1.summary()

In [ ]:
# Train model
test_history1 = test_model1.fit(train_event_inputs, train_event_labels, validation_split=0.33,
                                batch_size =100, epochs=25, verbose = 2)

In [ ]:
# Plotting accuracy over epochs
fig,ax=plt.subplots(1,2, figsize=(15,5))
ax[0].plot(test_history1.history['binary_accuracy'], linewidth=3)
ax[0].plot(test_history1.history['val_binary_accuracy'], linewidth=3)

# Setting axis and labels
ax[0].set_xlabel('Epoch')
ax[0].set_ylabel('Accuracy')
ax[0].set_title('Plot of Accuray over training Epoch')

ax[1].plot(test_history1.history['loss'], linewidth=3)
ax[1].plot(test_history1.history['val_loss'], linewidth=3)

# Setting axis and labels
ax[1].set_xlabel('Epoch')
ax[1].set_ylabel('Loss')
ax[1].set_title('Plot of Accuray over training Epoch')
plt.show()

#### Comment:
- We get good training curves and good validation accuracy, showing no overfitting, however,, the validation values are eratic and drop below training values, will combine with further layers to seperate curves more.

#### Model 2:
The track images show movement through space, each verticle line of pixels show trackes subsequent movement. We have implimented a LSTM layer to see if they can be some correlation found in the psuedo-causality described, we then use a convolutional layers for the featue learning. 

In [ ]:
# First input branch
x_input = Input(shape=(100, 80))

# LSTM layer, increased dimensions to then convolute
x_lstm   = keras.layers.LSTM(40, return_sequences=True)(x_input)
x_lstm   = tf.expand_dims(x_lstm, axis=-1)

# Convolutional and Pooling Layer for feature learning
x_conv2d = keras.layers.Conv2D(32, 3, padding='same')(x_lstm)
x_pool2d = keras.layers.MaxPool2D((2,2), strides = 2, padding='same')(x_conv2d)


# Second input branch
y_input = Input(shape=(100, 80))

# LSTM layer, increased dimensions to then convolute
y_lstm   = keras.layers.LSTM(40, return_sequences=True)(y_input)
y_lstm   = tf.expand_dims(y_lstm, axis=-1)

# Convolutional and Pooling Layer for feature learning
y_conv2d = keras.layers.Conv2D(32, 3, padding='same')(y_lstm)
y_pool2d = keras.layers.MaxPool2D((2,2), padding='same')(y_conv2d)

# Joining branches 
xy_l1 = concatenate([x_pool2d, y_pool2d], axis=-1)
xy_l2 = keras.layers.Conv2D(64, 2, padding='same')(xy_l1)

# Fully connected Neural Network
xy_l3 = keras.layers.Flatten()(xy_l2)
xy_l4 = keras.layers.Dense(128, activation = 'relu')(xy_l3)
xy_l5 = keras.layers.Dropout(0.5)(xy_l4)

# Output layer
output_layer = keras.layers.Dense(1)(xy_l5)


test_model2 = Model(inputs= (x_input,y_input), outputs=output_layer)

# Compile the network with binary crossentropy loss and adam optimiser with learning rate of 1.0
test_model2.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.001), 
                    loss      = tf.keras.losses.BinaryCrossentropy(from_logits=True), 
                    metrics   = [keras.metrics.BinaryAccuracy()])

test_model2.summary()

In [ ]:
# Train model
test_history2 = test_model2.fit((train_event_inputs[:,:,:,0], train_event_inputs[:,:,:,1]), train_event_labels, 
                          validation_split=0.33,batch_size =100, epochs=10, verbose = 2)

In [ ]:
# Plotting accuracy over epochs
fig,ax=plt.subplots(1,2, figsize=(15,5))
ax[0].plot(test_history2.history['binary_accuracy'], linewidth=3)
ax[0].plot(test_history2.history['val_binary_accuracy'], linewidth=3)

# Setting axis and labels
ax[0].set_xlabel('Epoch')
ax[0].set_ylabel('Accuracy')
ax[0].set_title('Plot of Accuray over training Epoch')

ax[1].plot(test_history2.history['loss'], linewidth=3)
ax[1].plot(test_history2.history['val_loss'], linewidth=3)

# Setting axis and labels
ax[1].set_xlabel('Epoch')
ax[1].set_ylabel('Loss')
ax[1].set_title('Plot of Accuray over training Epoch')
plt.show()

#### Comments:
- We see the training curves show good exponential trends, however, the validation data shows plateau and validation loss diverges, this shows the model is overfitting and not applicable to this data

#### Model 3:
Following model similar to that seen on paper 'fermilab-pub-16-082-nd.pdf' and using an inception model originally introduced in Going Deeper With Convolutions". We recreate similar to see its computation power and if there is any architecture we can adopt. An inception layer is a combination of convolutional layers (namely, 1x1 Convolutional layer, 3x3 Convolutional layer, 5x5 Convolutional layer) with their output filter banks concatenated into a single output vector forming the input of the next stage. Here we impliment a inception module with dimensionality reduction by including additional 1x1 convolutional layers.

In [ ]:
# Setting up Inception architecture
def inception_module(layer_in, k1, k3, k5, kp):
    # 1x1 conv
    conv1 = Conv2D(k1, (1,1), padding='same', activation='relu')(layer_in)
    
    # 3x3 conv
    conv3 = Conv2D(k3, (1,1), padding='same', activation='relu')(layer_in)
    conv3 = Conv2D(k3, (3,3), padding='same', activation='relu')(conv3)
    
    # 5x5 conv
    conv5 = Conv2D(k5, (1,1), padding='same', activation='relu')(layer_in)
    conv5 = Conv2D(k5, (5,5), padding='same', activation='relu')(conv5)
    
    # Pooling
    convp = MaxPooling2D((2,2), strides = 1, padding='same')(layer_in)
    convp = Conv2D(kp, (1,1), padding='same', activation='relu')(convp)
 
    # concatenate filters, assumes filters/channels last
    layer_out = concatenate([conv1, conv3, conv5, convp], axis=-1)

    return layer_out

####################################################################

# First input branch
x_input = Input(shape=(100, 80, 1))

# First convolutional layer of 32, 7x7 kernals; Pooling 3x3 with padding and Batch Normalisation
x_l1 = Conv2D(32, (7,7),strides= 2, padding='same', activation='relu')(x_input)
x_l2 = MaxPooling2D((3, 3),strides= 2, padding='same')(x_l1)
x_l3 = keras.layers.BatchNormalization()(x_l2)

# Second convolutional layer of 32, 1x1 kernals
x_l4 = Conv2D(32, (1,1), strides= 2, padding='same', activation='relu')(x_l3)

# Third convolutional layer of 64, 3x3 kernals
x_l5 = Conv2D(64, (3,3),strides= 2, padding='same', activation='relu')(x_l4)
x_l6 = keras.layers.BatchNormalization()(x_l5)
x_l7 = MaxPooling2D((3, 3),strides= 2, padding='same')(x_l6)

# Inception modules and pooling, kernals followed from paper above
x_l8 = inception_module(x_l7,64, 96, 16,32)
x_l9 = inception_module(x_l8, 128, 128, 32, 64)
x_l10 = MaxPooling2D((3, 3),strides= 2, padding='same')(x_l9)
x_l11 = inception_module(x_l10, 128, 128, 32, 64)

####################################################################

# Second input branch
y_input = Input(shape=(100, 80, 1))

# First convolutional layer of 32, 7x7 kernals; Pooling 3x3 with padding and Batch Normalisation
y_l1 = Conv2D(32, (7,7),strides= 2, padding='same', activation='relu')(y_input)
y_l2 = MaxPooling2D((3, 3),strides= 2, padding='same')(y_l1)
y_l3 = keras.layers.BatchNormalization()(y_l2)

# Second convolutional layer of 32, 1x1 kernals
y_l4 = Conv2D(32, (1,1), strides= 2, padding='same', activation='relu')(y_l3)

# Third convolutional layer of 64, 3x3 kernals
y_l5 = Conv2D(64, (3,3),strides= 2, padding='same', activation='relu')(y_l4)
y_l6 = keras.layers.BatchNormalization()(y_l5)
y_l7 = MaxPooling2D((3, 3),strides= 2, padding='same')(y_l6)

# Inception modules and pooling, kernals followed from paper above
y_l8 = inception_module(y_l7, 64, 96, 16, 32)
y_l9 = inception_module(y_l8, 128, 128, 32, 64)
y_l10 = MaxPooling2D((3, 3),strides= 2, padding='same')(y_l9)
y_l11 = inception_module(y_l10, 128, 128, 32, 64)

####################################################################

# Joining branches of network together
xy_l1 = concatenate([x_l11, y_l11], axis=-1)
xy_l2 = inception_module(xy_l1, 192, 96, 16, 64)
xy_l3 =  keras.layers.AveragePooling2D((6,5),padding='same')(xy_l2)

# Fully connected neural network
xy_l4 = keras.layers.Flatten()(xy_l3)
xy_l5 = keras.layers.Dense(64, activation='relu')(xy_l4)
xy_l6 = keras.layers.Dropout(0.3)(xy_l5)

# Output layer
output_layer = keras.layers.Dense(1)(xy_l6)

# Create model
test_model3 = Model(inputs= (x_input, y_input), outputs=output_layer)

# Compile the network with binary crossentropy loss and adam optimiser with learning rate of 1.0
test_model3.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.001), 
                    loss      = tf.keras.losses.BinaryCrossentropy(from_logits=True), 
                    metrics   = [keras.metrics.BinaryAccuracy()])

test_model3.summary()

In [ ]:
# Train model
test_history3 = test_model3.fit((train_event_inputs[:,:,:,0], train_event_inputs[:,:,:,1]), train_event_labels, 
                          validation_split=0.33,batch_size=100, epochs=15, verbose = 2)

In [ ]:
# Plotting accuracy over epochs
fig,ax=plt.subplots(1,2, figsize=(15,5))
ax[0].plot(test_history3.history['binary_accuracy'], linewidth=3)
ax[0].plot(test_history3.history['val_binary_accuracy'], linewidth=3)

# Setting axis and labels
ax[0].set_xlabel('Epoch')
ax[0].set_ylabel('Accuracy')
ax[0].set_title('Plot of Accuray over training Epoch')

ax[1].plot(test_history3.history['loss'], linewidth=3)
ax[1].plot(test_history3.history['val_loss'], linewidth=3)

# Setting axis and labels
ax[1].set_xlabel('Epoch')
ax[1].set_ylabel('Loss')
ax[1].set_title('Plot of Accuray over training Epoch')
plt.show()

#### Comment:
- We get good validation accuracy at the begining comparitively, however, model wildly over fits and no characteristic learning curve, the inception module was taken and similar architecture was used on a simplified model, with dropout layers.

## Task 1: Binary Classifier:

### Training Binary Convolutional Model:
Model decided is

In [ ]:
# Setting up Inception architecture
def inception_module(layer_in, k1, k3, k5, kp):
    # 1x1 conv
    conv1 = Conv2D(k1, (1,1), padding='same', activation='relu')(layer_in)
    
    # 3x3 conv
    conv3 = Conv2D(k3, (1,1), padding='same', activation='relu')(layer_in)
    conv3 = Conv2D(k3, (3,3), padding='same', activation='relu')(conv3)
    
    # 5x5 conv
    conv5 = Conv2D(k5, (1,1), padding='same', activation='relu')(layer_in)
    conv5 = Conv2D(k5, (5,5), padding='same', activation='relu')(conv5)
    
    # Pooling
    convp = MaxPooling2D((2,2), strides = 1, padding='same')(layer_in)
    convp = Conv2D(kp, (1,1), padding='same', activation='relu')(convp)
 
    # concatenate filters, assumes filters/channels last
    layer_out = concatenate([conv1, conv3, conv5, convp], axis=-1)

    return layer_out

####################################################################

# Input branch
inputs = Input(shape=(100, 80, 2))

# First, Seperable convolutional layer of 32, 7x7 kernals; Pooling 2x2 and Spatial dropout for overfitting
conv1 = keras.layers.SeparableConv2D(32, 7, activation='relu', input_shape=(100,80,2))(inputs)
pool1 = keras.layers.MaxPooling2D((2, 2), strides = 2)(conv1)
drop1 = keras.layers.SpatialDropout2D(0.4)(pool1)

# Second convolutional layer of 32, 3x3 kernals; Pooling 2x2  and dropout for overfitting
conv2 = keras.layers.Conv2D(64, 3, activation='relu', input_shape=(100,80,2))(drop1)
pool2 = keras.layers.MaxPooling2D((2, 2), strides = 2)(conv2)
drop2 = keras.layers.Dropout(0.4)(pool2)

# Inception layer
incp1 = inception_module(drop2,64, 96, 16,32)

# Third convolutional layer of 64, 3x3 kernals 
conv3 = keras.layers.Conv2D(64, 3, activation='relu')(incp1)
drop3 = keras.layers.Dropout(0.7)(conv3)

# Fully connected Neural Network
flatn = keras.layers.Flatten()(drop3)
dens1 = keras.layers.Dense(128, activation='relu')(flatn)
drop4 = keras.layers.Dropout(0.8)(dens1)
dens2 = keras.layers.Dense(64, activation='relu')(drop4)

# We have a dropout layer to prevent overfitting
drop = keras.layers.Dropout(0.8)(dens2)

# Create output layer
output_layer = keras.layers.Dense(1, activation = 'sigmoid')(drop)

# Create model
model = Model(inputs= inputs, outputs=output_layer)

# Compile the network with binary crossentropy loss and adam optimiser with learning rate of 1.0
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.001), 
                    loss      = tf.keras.losses.BinaryCrossentropy(from_logits=False), 
                    metrics   = [keras.metrics.BinaryAccuracy()])

model.summary()

In [ ]:
# history = model.fit((train_event_inputs[:,:,:,0], train_event_inputs[:,:,:,1]), train_event_labels, 
#                     validation_split=0.33, batch_size =100, epochs=15, verbose = 2)

history = model.fit( train_event_inputs, train_event_labels, 
                     validation_split=0.33, batch_size =100, epochs=35, verbose = 2)

In [ ]:
# Plotting accuracy over epochs
fig,ax=plt.subplots(1,2, figsize=(15,5))
ax[0].plot(history.history['binary_accuracy'], linewidth=1, label = 'binary_accuracy')
ax[0].plot(history.history['val_binary_accuracy'],'--', linewidth=1, label = 'val_binary_accuracy')

# Setting axis and labels
ax[0].set_xlabel('Epoch')
ax[0].set_ylabel('Accuracy')
ax[0].set_title('Plot of Accuray over training Epoch')
ax[0].legend()
 
ax[1].plot(history.history['loss'], linewidth=1,label = 'loss')
ax[1].plot(history.history['val_loss'],'--',linewidth=1,label = 'val_loss')

# Setting axis and labels
ax[1].set_xlabel('Epoch')
ax[1].set_ylabel('Loss')
ax[1].set_title('Plot of Loss over training Epoch')
ax[1].legend()

plt.show()

### Testing Model Performance:

In [ ]:
# Create testing data in same way as training data
event  = np.array(data[10]['cvnmap'], dtype='uint8')
event_l= np.array(data[10]['neutrino']['interaction'],dtype='uint8')

for i in range(11,21):
    event  = np.concatenate(( event   , np.array(data[i]['cvnmap'], dtype = 'uint8') ))  
    event_l= np.concatenate(( event_l , np.array(data[i]['neutrino']['interaction'], dtype='uint8') )) 

# Normalisation of images
# event /= np.max(event)
# event /= np.max(event)
    
# Test data with unequal data
test_event_inputs = event.reshape((event.shape[0],2,100,80)).transpose(0,2,3,1)
test_event_labels = np.where( event_l > 3, 0, 1).astype(bool)

In [ ]:
# Equalise test data and store seperately
test_event_inputs_eq, test_event_labels_eq = ClassEqualised(test_event_inputs, test_event_labels)

# Test accuracy of trained network
test_loss, test_acc = model.evaluate(test_event_inputs, test_event_labels, verbose=0)
test_loss_eq, test_acc_eq = model.evaluate(test_event_inputs_eq, test_event_labels_eq, verbose=0)

print('The accuracy of the network on unbalanced data is', test_acc)
print('The accuracy of the network on balanced data is', test_acc_eq)

#### Comment:
- We see model losses accuracy when test on whole data set, when equalised we recover 83% accuracy, we will use `model.predict` to further explore the data

In [ ]:
# Model.predict produce array of predictions (percentage of certainties) 
y_score = model.predict(test_event_inputs, batch_size = 100)
# True values are the labels
y_true  = test_event_labels

# Array of equalised data prediction
y_score_eq = model.predict(test_event_inputs_eq, batch_size = 100)
# True values are the labels
y_true_eq  = test_event_labels_eq

In [ ]:
# Create class dictionary
test_classes = Classes(test_event_inputs, test_event_labels)

# Plotting Histogram of predictions
fig,ax=plt.subplots(1,2, figsize=(15,5))

# Non-equalised Predictions
ax[0].hist(test_event_labels.astype(int), label = 'Labels')
ax[0].hist(y_score, label = 'Predictions')

# Setting axis and labels
ax[0].set_xlabel('Predicted Values')
ax[0].set_ylabel('Frequencies')
ax[0].set_title('Histogram of Predictions on Unbalanced data')
ax[0].legend()

# Equalised data prediction
ax[1].hist(test_event_labels_eq.astype(int), label = 'Labels')
ax[1].hist(y_score_eq, label = 'Predictions')

# Setting axis and labels
ax[1].set_xlabel('Predicted Values')
ax[1].set_ylabel('Frequencies')
ax[1].set_title('Histogram of Predictions on Balanced data')
ax[1].legend()

plt.show()

#### Comment:
- These histograms show bias in the predictions for the unbalanced data as far more predictions are given as true compared to false; this aligns with the drop in predictions as a large percentage of these ones are false positives. 

#### ROC and Threshold Exploration:
A receiver operating characteristic curve, ROC curve, is a graphical plot that illustrates the diagnostic ability of a binary classifier system as its discrimination threshold is varied. The ROC curve summarizes the trade-off between the true positive rate and false positive rate for a predictive model using different probability thresholds

In [ ]:
# Roc curve data and false positives/ true positive rate, using sklearn function
fpr, tpr, threshold  =  metrics.roc_curve(y_true, y_score)
fpr_eq, tpr_eq, threshold_eq  =  metrics.roc_curve(y_true_eq, y_score_eq)

# Plotting ROC and rates for unbalanced data
fig,ax=plt.subplots(1,2, figsize=(15,5))
ax[0].plot( fpr, tpr, label = 'ROC curve')
ax[0].plot( threshold, fpr,'--', label = 'True False rate')
ax[0].plot( threshold, tpr, ':', label = 'True Positive rate')
ax[0].set_xlim(0,1)

# Setting axis and labels
ax[0].set_xlabel('False Positive Rate/ Threshold')
ax[0].set_ylabel('True Positive Rate/ Rates')
ax[0].set_title('Plot of ROC curve and False/True Positive rate for Unbalanced data')
ax[0].legend()

# Plot for balanced data
ax[1].plot( fpr_eq, tpr_eq, label = 'ROC curve')
ax[1].plot( threshold_eq, fpr_eq,'--', label = 'True False rate')
ax[1].plot( threshold_eq, tpr_eq, ':', label = 'True Positive rate')
ax[1].set_xlim(0,1)

# Setting axis and labels
ax[1].set_xlabel('False Positive Rate/ Threshold')
ax[1].set_ylabel('True Positive Rate/ Rates')
ax[1].set_title('Plot of ROC curve and False/True Positive rate for Balanced data')
ax[1].legend()

plt.show()

#### Comments:
- Our curve shows shape close to the top-left corner which shows good classification; however, the non-symmetric shape shows the false positive rate increases faster than the true positive rate, showing overall bias in classifier to predict false positive. 

#### Threshold Testing:
We look at accuracy variation over different thresholds, and use this to set the unbias threshold to use

In [ ]:
# Number of thresholds
Nt = 100

# Initialise threshold, accuracy and rate arrays
thresholds = np.linspace(0,1,Nt)
accuracys, accuracys_eq = np.zeros(Nt), np.zeros(Nt)
rate, rate_eq = np.zeros([Nt,2]), np.zeros([Nt,2])

# Loop to test cooresponding accuracy for each threshold
for i, thres in enumerate(thresholds):
    
    # Predictions arrays are given by changing 'score' to binary labels using threshold(below threshold gives a 0)
    y_pred    = np.where( y_score < thres, 0, 1).astype(bool)
    y_pred_eq = np.where( y_score_eq < thres, 0, 1).astype(bool)
    
    # Use sklearn import to calculate accuracy
    test_acc    = metrics.accuracy_score(y_true, y_pred, normalize=True)
    test_acc_eq = metrics.accuracy_score(y_true_eq, y_pred_eq, normalize=True)

    # Generate confusion matrix to find true positive, false positive, true negative, false negative values
    con = metrics.confusion_matrix(y_true, y_pred)
    con_eq = metrics.confusion_matrix(y_true_eq, y_pred_eq)
    
    # Store correct and wrong predictions
    rate[i][0], rate[i][1] = (con[0,0]+con[1,1]), (con[1,0]+con[0,1])
    rate_eq[i][0], rate_eq[i][1] = (con_eq[0,0]+con_eq[1,1]), (con_eq[1,0]+con_eq[0,1])
    
    # Store accuracys
    accuracys[i]    = test_acc
    accuracys_eq[i] = test_acc_eq

In [ ]:
# Plotting accuracy against threshold
fig,ax=plt.subplots(1,1, figsize=(10,5))

ax.plot(thresholds, accuracys, label = 'Unbalanced data')
ax.plot(thresholds, accuracys_eq, '--', label = 'Balanced data' )

# Setting axis and labels
ax.set_xlabel('Prediction Threshold')
ax.set_ylabel('Accuracy')
ax.set_title('Plot of Accuray over prediction Threshold')
ax.legend()

plt.show()

#### Comment:
- For the unbalanced data this bias is evident in Fig. 10 as reducing the threshold, such that predictions all become false, increases accuracy as the overall unbalanced distribution has approximately 88% false. This is regardless of classifier performance and the reverse is seen when increasing threshold to one. Comparatively, for balanced data the accuracy for a threshold of both one and zero gives 50% giving a more accurate baseline for analysis. We can use the intersection of the two graphs to set an unbiases threshold for analysis of the meta data as increased accuracy beyond the intersection is only due to any bias in data set. Thus, the threshold used in further analysis is 0.5 which indicates good separation between classification; this is further supported by the shape which shows high accuracy over a wide threshold range implying correct classifications and separation over range

## Task 2: Testing Meta Data:

#### Set up dictionaries:
Create dictionaries holding corresponding scores and corect labels for each meta data strata to easy analysis.

In [ ]:
# Initialise dictionaries
event_labels_dic, event_scores_dic = {},{}
event_labels_dic_eq, event_scores_dic_eq = {},{}
# Loops through all data sets
for i in tqdm(range(1,30)):
    # Produce images to test on model
    event        = np.array(data[i]['cvnmap'], dtype='uint8')    
    event_inputs = event.reshape(( event.shape[0],2,100,80 )).transpose(0,2,3,1)
    
    # Produce corresponding labels
    event_l      = np.array(data[i]['neutrino']['interaction'],dtype='uint8')
    event_labels = np.where( event_l > 3, 0, 1).astype(bool)
    
    # Predict scores fof images produced, also creates classed equal scores and labels
    event_scores = model.predict(event_inputs, batch_size = 200)
    event_scores_eq , event_labels_eq = ClassEqualised(event_scores, event_labels)

    # Add to labels and scores dictionary, using kwargs of the function to add energies in bins of 5eV
    event_labels_dic = Dictionary(event_labels_dic, event_labels, i, nuenergy = 5, lepenergy = 5, energy = 5)
    event_scores_dic = Dictionary(event_scores_dic, event_scores, i, nuenergy = 5, lepenergy = 5, energy = 5)
    
    # Add to equalised labels and scores dictionary, using kwargs of the function to add energies in bins of 5eV
    event_labels_dic_eq = Dictionary(event_labels_dic_eq, event_labels_eq, i, nuenergy = 5, lepenergy = 5, energy = 5)
    event_scores_dic_eq = Dictionary(event_scores_dic_eq, event_scores_eq, i, nuenergy = 5, lepenergy = 5, energy = 5) 

In [ ]:
# Ensure interaction key contains all interaction types to make looping easier
for i in range(17):
    if i in event_labels_dic['interaction']:
        None
    else:
        event_labels_dic['interaction'][i] = []
        event_scores_dic['interaction'][i] = [] 

    if i in event_labels_dic_eq['interaction']:
        None
    else:
        event_labels_dic_eq['interaction'][i] = []
        event_scores_dic_eq['interaction'][i] = []        

### Interaction Accuracy Test:
 Testing betweenw interaction type:
- QE: Clean event, normally just two tracks
- RES: Something in the middle
- DIS: Messy event potentially many tracks and showers

In [ ]:
def Interaction_acc(Labels_dic, Scores_dic, thres, int_type):
    ''' Function to calculate accuracy of given interaction type.
    
    Inputs:
    Labels_dic - Dictionary containing all labels in meta data strata
    Scores_dic - Dictionary containing all scores in meta data strata
    thres      - Threshold from which scores are turned assigned 0/1
    int_type   - Interaction type, 0: QE, 1: DIS, 2:Res, 4:Others
    
    Return:
    Accuracy- Accuracy of Interaction type
    '''
    
    # List comprehension to extract data from dictionary for thee interaction type given
    Labels    = np.array( [ value for i in range(3) for value in Labels_dic['interaction'][int_type + 4*i]] )
    Scores    = np.array( [ value for i in range(3) for value in Scores_dic['interaction'][int_type + 4*i]] )
    
    # Use threshold to produce predictions and calculate accuracy
    Prediction = np.where( Scores < thres, 0, 1).astype(bool)
    Accuracy   = metrics.accuracy_score(Labels, Prediction, normalize=True)
    
    return Accuracy

In [ ]:
# kNuElectronElastic = 12
# kNC =13  
# kCosmic =14      
# kOther =15   
# kNIntType=16 

# Set threshold discussed above
thres = 0.5

# Calculate accuracy for each interaction type
QE_acc     = Interaction_acc(event_labels_dic, event_scores_dic, thres, 0)
Res_acc    = Interaction_acc(event_labels_dic, event_scores_dic, thres, 1)
DIS_acc    = Interaction_acc(event_labels_dic, event_scores_dic, thres, 2)
Other_acc  = Interaction_acc(event_labels_dic, event_scores_dic, thres, 3)

# Store Accuracys
Type_keys = np.array(['QE_acc', 'DIS_acc', 'Res_acc', 'Other_acc'])
Type_acc  = np.array([QE_acc, DIS_acc, Res_acc, Other_acc])

print('QE accuracy:',QE_acc)
print('Res accuracy:',Res_acc)
print('DIS accuracy:',DIS_acc)
print('Other accuracy:',Other_acc)

In [ ]:
# Plot of interaction types accuracy
fig, ax = plt.subplots(1,1, figsize=(20,10))
ax.bar(Type_keys, Type_acc)
ax.set_title('Accuracy of Interaction Type')
ax.set_xlabel('Interaction Type')
ax.set_ylabel('Accuracy')

plt.show()

#### Comment:
- This is the expected result as the QE are clearer two track interaction thus are more easily distinguished compared to DIS which are messier potentially with many tracks and showers. RES events have an accuracy in between as their tracks are less degradation then DIS but still less clear than QE events

In [ ]:
# Initialise threshold array
Nt = 100
thresholds = np.linspace(0,1,Nt)

# Initialise accuracy arrays
QE_acc_thres, DIS_acc_thres, Res_acc_thres, Other_acc_thres = np.zeros(Nt), np.zeros(Nt), np.zeros(Nt), np.zeros(Nt)
QE_acc_thres_eq, DIS_acc_thres_eq, Res_acc_thres_eq, Other_acc_thres_eq = (np.zeros(Nt), np.zeros(Nt), 
                                                                           np.zeros(Nt), np.zeros(Nt))
# Calculate accuracy for all threshold values
for i, thres in enumerate(thresholds):
    # Calculate class equalised accuracis
    QE_acc_thres_eq[i]     = Interaction_acc(event_labels_dic_eq, event_scores_dic_eq, thres, 0)
    DIS_acc_thres_eq[i]    = Interaction_acc(event_labels_dic_eq, event_scores_dic_eq, thres, 1)
    Res_acc_thres_eq[i]    = Interaction_acc(event_labels_dic_eq, event_scores_dic_eq, thres, 2)
    Other_acc_thres_eq[i]  = Interaction_acc(event_labels_dic_eq, event_scores_dic_eq, thres, 3)
    
    # Calculate class non-equalised accuracis
    QE_acc_thres[i]     = Interaction_acc(event_labels_dic, event_scores_dic, thres, 0)
    DIS_acc_thres[i]    = Interaction_acc(event_labels_dic, event_scores_dic, thres, 1)
    Res_acc_thres[i]    = Interaction_acc(event_labels_dic, event_scores_dic, thres, 2)
    Other_acc_thres[i]  = Interaction_acc(event_labels_dic, event_scores_dic, thres, 3)

In [ ]:
# Plot accuracies variation over threshold
fig, ax = plt.subplots(1,2, figsize=(15,5))

ax[0].plot(thresholds, QE_acc_thres,          linewidth=0.75,  label = 'QE Interaction')
ax[0].plot(thresholds, Res_acc_thres,   '--', linewidth=0.75,  label = 'Res Interaction')
ax[0].plot(thresholds, DIS_acc_thres,    ':', linewidth=0.75,  label = 'DIS Interaction')
ax[0].plot(thresholds, Other_acc_thres, '-.', linewidth=0.75,  label = 'Other Interaction')

# Setting axis and labels
ax[0].set_xlabel('Predictive Threshold')
ax[0].set_ylabel('Accuracy')
ax[0].set_title('Plot of Interaction accuracy over  threshold for balanced data ')
ax[0].legend()

# Balanced data plot
ax[1].plot(thresholds, QE_acc_thres_eq,          linewidth=1,  label = 'QE Interaction')
ax[1].plot(thresholds, Res_acc_thres_eq,   '--', linewidth=1,  label = 'Res Interaction')
ax[1].plot(thresholds, DIS_acc_thres_eq,    ':', linewidth=1,  label = 'DIS Interaction')
ax[1].plot(thresholds, Other_acc_thres_eq, '-.', linewidth=1,  label = 'Other Interaction')

# Setting axis and labels
ax[1].set_xlabel('Predictive Threshold')
ax[1].set_ylabel('Accuracy')
ax[1].set_title('Plot of Interaction accuracy over threshold for unbalanced data ')
ax[1].legend()


plt.show()

#### Comments:
- To ensure this behaviour is not a feature of the threshold chosen, the accuracy for the interactions is then plotted over a range of frequencies. Graph shows same characteristic shape of the unbalanced data , however, here the interaction type is split and show same ordering as above over the whole range, thus the correlation holds over the whole range

#### Function for testing other Meta Data types:

In [ ]:
def Meta_Accuracy(meta_data, Labels_dic, Scores_dic, thres):
    ''' Function to calculate accuracy over the sub strata of given meta data type.
    
    Inputs:
    Meta_data  - String of meta data; Accuracy calculated for over substrata of that meta data
    Labels_dic - Dictionary contain Labels in meta data strata
    Scores_dic - Dictionary contain Scores in meta data strata
    thres      - Threshold in which Scores are converted to predictions
    
    Return:
    meta_data_keys - Array containing meta data keys
    meta_acc       - Array containing accuracies for each meta data key
    '''
    # Extract keys of meta data and sort
    meta_data_keys = np.array( list(Labels_dic[meta_data].keys() ))
    meta_data_keys.sort()

    # Initialise array to hold accuracies
    meta_acc = np.zeros(len(meta_data_keys))
    
    # For each meta data key calculate the accuracy
    for i,v in enumerate(meta_data_keys):
        
        # Retrieve Labels and Scores corresponding for meta data key
        Labels = np.array( Labels_dic[meta_data][v] )
        Scores = np.array( Scores_dic[meta_data][v] )
        
        # Turns score to prediction and calculate accuracy
        Prediction = np.where( Scores < thres, 0, 1).astype(bool)
        Accuracy   = metrics.accuracy_score(Labels, Prediction, normalize=True)
        
        # Store accuracy for key
        meta_acc[i] = Accuracy       
       
    return meta_data_keys, meta_acc

### Energy Accuracy Test:

In [ ]:
# Energy accuracies
lepenergy_keys,  lepenergy_acc  = Meta_Accuracy('lepenergy',  event_labels_dic, event_scores_dic, 0.5)
nuenergy_keys,   nuenergy_acc   = Meta_Accuracy('nuenergy', event_labels_dic, event_scores_dic, 0.5)
energy_keys,     energy_acc     = Meta_Accuracy('energy',  event_labels_dic, event_scores_dic, 0.5)

In [ ]:
# Energy accuracies in bins of 5eV 
lepenergy5_keys,  lepenergy5_acc  = Meta_Accuracy('lepenergy5',  event_labels_dic, event_scores_dic, 0.5)
nuenergy5_keys,   nuenergy5_acc   = Meta_Accuracy('nuenergy5', event_labels_dic, event_scores_dic, 0.5)
energy5_keys,     energy5_acc     = Meta_Accuracy('energy5',  event_labels_dic, event_scores_dic, 0.5)

In [ ]:
def fit(data):
    '''Function to linearly fit data'''
    fit, cvm = np.polyfit(data[0],  data[1], 1, cov='scaled')
    dfit = [np.sqrt(cvm[i,i]) for i in range(2)]  

    x   = np.linspace(0,np.max(data[0]),100)
    y   = fit[0]*x + fit[1]
    err = dfit[0]*x + dfit[1]
    
    return x, y, err

nu_x, nu_y, nu_err = fit((nuenergy5_keys,  nuenergy5_acc))
lep_x, lep_y, lep_err = fit((lepenergy5_keys[:-1],  lepenergy5_acc[:-1]))
energy_x, energy_y, energy_err = fit((energy5_keys[:-1],  energy5_acc[:-1]))

In [ ]:
# Plot of energy accuracies
fig, ax = plt.subplots(1,3, figsize=(25,5))

ax[0].plot(energy5_keys[:-1], energy5_acc[:-1],'.')
ax[0].plot(energy_x,  energy_y,'--')
ax[0].errorbar(energy_x[::5], energy_y[::5], yerr=energy_err[::5], color='black', linestyle='None', capsize=2)

# Setting axis and labels
ax[0].set_xlabel('Energy, eV')
ax[0].set_ylabel('Accuracy')
ax[0].set_title('Plot of Accuracy over Lepton + Netrino Energy')

ax[1].plot(lepenergy5_keys[:-1],  lepenergy5_acc[:-1],'.')
ax[1].plot(lep_x,  lep_y,'--')
ax[1].errorbar(lep_x[::5], lep_y[::5], yerr=lep_err[::5], color='black', linestyle='None', capsize=2)

# Setting axis and labels
ax[1].set_xlabel('Energy,eV')
ax[1].set_ylabel('Accuracy')
ax[1].set_title('Plot of Accuracy over Lepton Energy')

ax[2].plot(nuenergy5_keys,   nuenergy5_acc,'.')
ax[2].plot(nu_x,  nu_y,'-')
ax[2].errorbar(nu_x[::5], nu_y[::5], yerr=nu_err[::5], color='black', linestyle='None', capsize=2)
# Setting axis and labels

ax[2].set_xlabel('Energy, eV')
ax[2].set_ylabel('Accuracy')
ax[2].set_title('Plot of Accuracy over Netrino Energy')

plt.show()

#### Comment:
- the higher the energy the clearer the tract as greater ‘exposure’ in the image and high energy particles are more penetrative therefore will have long and more distinct tracks, in addition, there is less overlap visual in differences interaction type at high energy. This allows for better feature extraction and correlation of underlying image topology to given interaction. 

### Testing other meta data accuracy:

In [ ]:
# Other meta data accuracies
finalstate_keys, finalstate_acc           = Meta_Accuracy('finalstate',      event_labels_dic, event_scores_dic, 0.5)
finalstateprong_keys, finalstateprong_acc = Meta_Accuracy('finalstateprong', event_labels_dic, event_scores_dic, 0.5)
parent_keys,          parent_acc          = Meta_Accuracy('parent',          event_labels_dic, event_scores_dic, 0.5)
evt_keys,             evt_acc             = Meta_Accuracy('evt',             event_labels_dic, event_scores_dic, 0.5)
particles_keys,       particles_acc       = Meta_Accuracy('particles',       event_labels_dic, event_scores_dic, 0.5)

In [ ]:
# Encode labels for finals states
FinalState_keys = np.array([FinalState(i).name for i in finalstate_keys])

# Plot final state accuracies
fig, ax = plt.subplots(1,1, figsize=(15,10))

# Plot accuracy bars
ax.bar(FinalState_keys, finalstate_acc)
ax.tick_params(labelrotation=90)

# Set labels and titles 
ax.set_title('Accuracy of Final State Type')
ax.set_xlabel('Interaction Type')
ax.set_ylabel('Accuracy')

plt.show()

print('Average for NuMu final states',np.mean(finalstate_acc[:16]),'\nAverage for none NuMu final states', np.mean(finalstate_acc[16:]))

#### Comment:
- For mu neutrino final states have lower accuracies than that of the other final state; when averaged we find mu neutrinos final states have an average accuracy of 74.2% compared to the other final states which have average accuracy of 88.6%. This could be due to the ubiquity of mu neutrinos there is a large number and thus wide variety of interaction recorded, therefore, greater overlap with other interaction types which leads to incorrect categorisation.



In [ ]:
# Plot final state accuracies
fig, ax = plt.subplots(2, 2, figsize=(25,10))

ax[0,0].bar(finalstateprong_keys, finalstateprong_acc)
ax[0,0].set_title('Accuracy for Final State prong')
ax[0,0].set_xlabel('Final State prong')
ax[0,0].set_ylabel('Accuracy')

ax[0,1].bar(parent_keys,  parent_acc)
ax[0,1].set_title('Accuracy for Parent type')
ax[0,1].set_xlabel('Parent')
ax[0,1].set_ylabel('Accuracy')

ax[1,0].bar(evt_keys,   evt_acc)
ax[1,0].set_title('Accuracy for Event type')
ax[1,0].set_xlabel('Event type')
ax[1,0].set_ylabel('Accuracy')


ax[1,1].bar(particles_keys,   particles_acc)
ax[1,1].set_title('Accuracy for Particle type')
ax[1,1].set_xlabel('Particle')
ax[1,1].set_ylabel('Accuracy')


plt.show()

#### Comment:
- Analysing the other meta data shows no other dependency on accuracy as can be seen in the data in Fig.15. There is not trend in the data, however, this may require processing to highlight any trend, which could be done with further study.